<h1> Feedforward Fully-connected Neuralnetwork </h1>


<h3>Proto</h3>
단순하게 학습정도만 구현할것임. 버전관리하면서 기능 추가해볼것  

batch_size : size of mini-batch INT  
n_iteration : number of iterations INT  
n_neurons : number of neurons to hidden layer INT    
n_layers : number of hidden layers INT    
learning_rate : learning rate FLOAT    
activation_function : activation function. [ 'sigmoid', 'relu' ]   
output_activation : activation function for last layer. [ 'sigmoid', 'relu', 'crossentropy', 'softmax' ]  
optimizer : gradient descent algorithm. [ 'minibatchSGD' ] 

In [4]:
import pandas as pd
import numpy as np

In [13]:
#preprocess

s = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'
df = pd.read_csv(s,
                 header=None,
                 encoding='utf-8')

X, y = df.iloc[:,:4], df.iloc[:,4]

def label_encoding(x):
    if x == 'Iris-setosa':
        return 1
    elif x == 'Iris-virginica':
        return 3
    else:
        return 2
    
y = list(map(label_encoding, y))

In [48]:
class feedforwardNN(object):
    def __init__(n_iteration = 100, n_neurons = 20, n_layers = 2, learning_rate = 0.1, activation_function = 'relu', output_activation = 'relu', optimizer = 'minibatchSGD', random_state = 42):
        self.n_iteration = n_iteration
        self.n_layers = n_layers
        self.n_neurons = n_neurons
        self.activation_function = activation_function
        self.learning_rate = learning_rate
        self.output_activation = output_activation
        self.optimizer = optimizer
        self.random_state = random_state
        
    def fit(self, X, y):   
        def sigmoid(X):
            return 1 / (1 +np.exp(-x))
        def sigmoid_prime(X):
            return sigmoid(X)(1-sigmoid(X))
        def relu(X):
            return 0 if X <= 0 else X
        def step_function(X):
            return 0 if X <= 0 else X
        def cross_entropy(y_hat, y):
            #add delta to prevent y_hat to be zero
            delta = 1e-7 
            return -np.sum(y*np.log(y_hat+delta))
        def softmax(x):
            #sub max(x) to prevent overflow
            x = np.exp(x - np.max(x))
            return x / x.sum(axis=0)
        if isinstance(X, pd.DataFrame):
            X = X.to_numpy()
        assert X.shape[0] = y.shape[0], "X and y don't match."
        rgen = np.random.RandomState(self.random_state)
        def create_weight_and_bias(self):
            self.w_ = []
            self.b_ = []
            self.w_.append(rgen.normal(loc = .0, scale = .01, size = X.shape[1] * self.n_neurons).reshape(X.shape[1], self.n_neurons))
            self.b_.append(rgen.normal(loc = .0, scale = .01, size = self.n_neurons))
            for _ in range(self.n_layers - 1):
                self.w_.append(rgen.normal(loc = .0, scale = .01, size = np.power(self.n_neurons,2)).reshape(self.n_neurons,self.n_neurons))
                self.b_.append(rgen.normal(loc = .0, scale = .01, size = self.n_neurons))
            self.w_.append(rgen.normal(loc = .0, scale = .01, size = self.n_neurons))
            self.b_.append(rgen.normal(loc = .0, scale = .01, size = 1))
            
        o = []
        o_prime = []
        self.cost_ = []
        
        def create_minibatches(self, y):
            indices = np.arange(len(y))
            if self.shuffle:
                rgen.shuffle(indices)
            cut = [x for x in range(0, len(indices), batch_size)] + [len(indices)]
            mini_batches = []
            for i in range(1,len(cut)):
                mini_batches.append([indices[cut[i-1]:cut[i]]])

            return mini_batches
        batches = create_minibatches(y)

        #forward propagation
        for _ in range(self.n_iteration):
            cost = []
            for batch in batches:
                X_batch = X[tuple(batch)]
                y_batch = y[tuple(batch)]
                y_hat = predict(X)
                
            
            
            self.cost_.append(np.mean(cost))
            return self
            

        
    def predict(self, X):
        assert isinstance(self.w_, np.ndarray), 'Fit weights and bias before predict'
        for i in range(self.n_layers-1):
            
            
        if self.output_activation == 'relu':
            return 
        
        if self.output_activation == 'sigmoid':
            return
        
        if self.output_activation == 'softmax':
            return
        
        if self.output_activation == 'crossentropy':
            return
        